# 3) Analyze and evaluate optimization output - TRAINING PROTOCOLS

This final notebook uses the `runs.pkl` file created in notebook 2 and it analyzes:

- the distance between different feature sets in the parameter space
- the distance between different feature sets in the feature space - training protocols
- the distance between different feature sets in the extracellular signals

In [ ]:
import pickle
import pandas as pd
import seaborn as sns
import sys
import shutil

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import matplotlib.pyplot as plt
from scipy.spatial import distance
import MEAutility as mu
import json
import time
import numpy as np
from pathlib import Path

from scipy.stats import kruskal, mannwhitneyu, wilcoxon

import multimodalfitting as mf

%matplotlib notebook

In [ ]:
save_fig = False
figure_folder = Path(".") / "figures_hay_ais"

if save_fig:
    figure_folder.mkdir(exist_ok=True)

In [ ]:
base_dir = Path("../..")

In [ ]:
colors_dict = {"soma": "C0",
               "all": "C1",
               "sections": "C2",
               "single": "C3"}
feature_sets = {"soma": "soma",
                "all": "extra",
                "sections": "extra",
                "single": "extra"}
figsize = (10, 7)

## Load GT params and optimization output

In [ ]:
# general
model_name = "hay_ais"
probe_type = "planar" # linear 

cell_models_folder = base_dir / "cell_models"
model_folder = cell_models_folder / model_name
probe_file = model_folder / "fitting" / "efeatures" / "probe_BPO.json"

In [ ]:
# change this with folder containing your pkl file
results_date = '220120'  # '211124' '220111' # 
result_folder = base_dir / "results" / results_date

In [ ]:
cell = mf.create_ground_truth_model(model_name=model_name,
                                    release=False)
cell_release = mf.create_ground_truth_model(model_name=model_name,
                                            release=True)

probe = mf.define_electrode(probe_file=probe_file)

param_names = [param.name for param in cell.params.values() if not param.frozen]

params_release = {}
for param in cell_release.params_by_names(param_names):
    params_release[param.name] = param.value

In [ ]:
protocol_for_eap = "IDrest_300"

In [ ]:
pkl_file_name = "runs.pkl"

In [ ]:
data = pickle.load(open(result_folder / pkl_file_name, 'rb'))
df_optimization = pd.DataFrame(data)
df_model = df_optimization.query(f"model == '{model_name}'")

In [ ]:
# set strategy column
df_model.loc[:, "strategy"] = df_model["extra_strategy"].values.copy()
df_model.loc[df_model["feature_set"] == "soma", "strategy"] = "soma"

In [ ]:
fig, ax = plt.subplots()
min_evals = 3000

keep_idxs = []
for idx, row in df_model.iterrows():
    if max(row["nevals"]) > min_evals:
        keep_idxs.append(idx)
        ax.plot(row["nevals"], 
                row["logbook"].select("min"),
                color=colors_dict[row["strategy"]],
                ls='-', 
                lw=0.8,
                alpha=0.75)
    else:
        ax.plot(row["nevals"], 
                row["logbook"].select("min"),
                color=colors_dict[row["strategy"]],
                ls='--', 
                lw=0.5,
                alpha=0.75)

ax.set_title("Min fitness")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_xlabel("Neval")
ax.set_ylabel("Min fitness")
ax.set_yscale('log')

# Load protocols and original features

In [ ]:
extra_kwargs = mf.utils.get_extra_kwargs()
extra_kwargs

In [ ]:
eva_extra = mf.create_evaluator(
    model_name=model_name,
    feature_set="extra",
    extra_strategy="all",
    protocols_with_lfp="IDrest_300",
    **extra_kwargs
)

In [ ]:
# check num features
for strategy in np.unique(df_model.strategy):
    extra_strategy = strategy if strategy in ["all", "single", "sections"] else None
    eva = mf.create_evaluator(
        model_name=model_name,
        feature_set=feature_sets[strategy],
        extra_strategy=extra_strategy,
        protocols_with_lfp="IDrest_300",
        **extra_kwargs
    )
    print(f"Strategy {strategy} --> num features {len(eva.fitness_calculator.objectives)}")

# Compute release responses

In [ ]:
responses_release = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), param_values=params_release)

In [ ]:
eap_release = mf.utils.calculate_eap(responses=responses_release, protocols=eva_extra.fitness_protocols, 
                                     protocol_name=protocol_for_eap, **extra_kwargs)

In [ ]:
features_release = {}
for obj in eva_extra.fitness_calculator.objectives:
    features_release[obj.features[0].name] = {}
    if len(obj.features) == 1:
        feat = obj.features[0]
        feat_value = feat.calculate_feature(responses_release)
        feat_score = feat.calculate_score(responses_release)
        if feat_value is None:
            print(f"{feat.name} cannot be computed: skipping")
            continue
        features_release[feat.name]["value"] = feat_value
    else:
        print(f"More than one feature for objective: {obj.name}")

In [ ]:
fig_gt_intra = mf.plot_responses(responses_release, color="k", return_fig=True)

In [ ]:
ax = mf.plot_eap(responses_release, eva_extra.fitness_protocols, probe,
                 protocol_name=protocol_for_eap, color="k")
fig_gt_extra = ax.get_figure()

In [ ]:
if save_fig:
    fig_gt_intra.savefig(figure_folder / "gt_intra.pdf")
    fig_gt_extra.savefig(figure_folder / "gt_extra.pdf")

# Compute and plot best responses

In [ ]:
max_feature_value = 50
opt_results = {}

In [ ]:
for strategy in np.unique(df_model.strategy):
    opt_results[strategy] = {}
    opt_df = df_model.query(f"strategy == '{strategy}'")
    best_idx = np.argmin(opt_df.best_fitness)
    params_sample = opt_df.iloc[best_idx]
    params_dict = {k: v for k, v in zip(param_names, params_sample.best_params)}
    opt_results[strategy]["best_fitness"] = params_sample.best_fitness
    opt_results[strategy]["best_params"] = params_dict
    print(f"{strategy} --  best fitness: {params_sample.best_fitness}")

In [ ]:
for strategy in np.unique(df_model.strategy):
    print(f"Simulating best '{strategy}'")
    responses = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), 
                                        param_values=opt_results[strategy]["best_params"])
    opt_results[strategy]["responses"] = responses
    eap = mf.utils.calculate_eap(responses=responses, protocols=eva_extra.fitness_protocols, 
                                 protocol_name=protocol_for_eap, **extra_kwargs)
    opt_results[strategy]["eap"] = eap   
    eap_dist = distance.cosine(eap_release.ravel(), eap.ravel())
    opt_results[strategy]["eap_dist"] = eap_dist

In [ ]:
for strategy in np.unique(df_model.strategy):
    responses = opt_results[strategy]["responses"]
    features_best = {}
    for obj in eva_extra.fitness_calculator.objectives:
        feat = obj.features[0]
        features_best[feat.name] = {}
        if len(obj.features) == 1:
            feat_value = obj.features[0].calculate_feature(responses)
            if feat_value is None:
                feat_value = max_feature_value
            features_best[feat.name]["value"] = feat_value
            if "MEA" not in feat.name:
                feat_score = np.abs(features_release[feat.name]["value"] - feat_value) / feat.exp_std
            else:
                feat_score = np.abs(distance.cosine(features_release[feat.name]["value"], feat_value))
            features_best[feat.name]["score"] = feat_score
        else:
            print(f"More than one feature for objective: {obj.name}")
    opt_results[strategy]["features"] = features_best

In [ ]:
# plot
for strategy in np.unique(df_model.strategy):
    responses_to_plot = [responses_release, opt_results[strategy]["responses"]]
    colors = ["k", colors_dict[strategy]]
    labels = ["GT", strategy.upper()]
    fig_extra_intra_single = mf.plot_multiple_responses(responses_to_plot, 
                                                        colors=colors, return_fig=True, 
                                                        labels=labels)
    ax_extra = mf.plot_multiple_eaps(responses_to_plot, 
                                     eva_extra.fitness_protocols, probe,
                                     protocol_name=protocol_for_eap, 
                                     colors=colors, labels=labels)

In [ ]:
for strategy in opt_results:
    print(f"Cosine dist {strategy}: {opt_results[strategy]['eap_dist']}")

In [ ]:
if save_fig:
    fig_extra_intra.savefig(figure_folder / "extra_intra.pdf")
    fig_extra_extra.savefig(figure_folder / "extra_extra.pdf")
    fig_extra_single_intra.savefig(figure_folder / "single_intra.pdf")
    fig_extra_single.savefig(figure_folder / "single_extra.pdf")

## Compare best-fitted models

In [ ]:
df_test = pd.DataFrame.from_dict(opt_results, orient="index")
df_test["strategy"] = df_test.index

## Compare parameters 

Here we normalize the parameters based on the boundaries and compute the relative difference to GT.

In [ ]:
param_json = model_folder / "parameters.json"

with param_json.open() as f:
    params = json.load(f)

param_boundaries = {}
for param in params:
    if "bounds" in param:
        if isinstance(param['sectionlist'], list):
            for sec in param['sectionlist']:
                param_boundaries[f"{param['param_name']}_{sec}"] = param["bounds"]
        else:
            sec = param['sectionlist']
            param_boundaries[f"{param['param_name']}_{sec}"] = param["bounds"]

# scale params_release by boundaries
params_release_norm = {}
for param_name, param_val in params_release.items():
    bounds = param_boundaries[param_name]
    param_norm = (param_val - bounds[0]) / (bounds[1] - bounds[0])
    params_release_norm[param_name] = param_norm

In [ ]:
params_release_norm

In [ ]:
feature_set_array = []
param_name_array = []
param_value_array = []
param_norm_array = []
release_value_array = []
release_norm_array = []
diff_with_release_array = []
section_array = []

for i, (index, opt) in enumerate(df_test.iterrows()):
    params_dict = opt.best_params
    
    for param_name, param_value in params_dict.items():
        feature_set_array.append(opt.strategy)
        param_name_array.append(param_name)
        param_value_array.append(param_value)
        section = param_name.split("_")[-1]
        if section == "segment":
            section = "ais"
        section_array.append(section)
        release_value_array.append(params_release[param_name])
        release_norm_array.append(params_release_norm[param_name])
        # compute norm value
        bounds = param_boundaries[param_name]
        param_norm = (param_value - bounds[0]) / (bounds[1] - bounds[0])
        param_norm_array.append(param_norm)
        diff_with_release_array.append(abs(param_norm - params_release_norm[param_name]))

        
df_params = pd.DataFrame({"strategy": feature_set_array, "param_name": param_name_array,
                          "param_value": param_value_array, "param_norm": param_norm_array, 
                          "release_value": release_value_array, "release_norm": release_norm_array,
                          "diff_release": diff_with_release_array, "section": section_array}) 

In [ ]:
# overall parameter diff
fig, ax = plt.subplots(figsize=figsize)
sns.boxenplot(data=df_params, y="strategy", x="diff_release", ax=ax)
ax.set_xlabel("Norm. param difference", fontsize=12)
ax.set_ylabel("Straregy", fontsize=12)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_title("All params", fontsize=15)


In [ ]:
for section in np.unique(df_params.section):
    fig, ax1 = plt.subplots(figsize=figsize)
    df_section = df_params.query(f"section == '{section}'")
    sns.barplot(data=df_section, y="param_name", x="diff_release", hue="strategy",
                orientation="horizontal", ax=ax1)
    ax1.set_xlabel("Norm. param difference", fontsize=12)
    ax1.set_ylabel("Param", fontsize=12)
    ax1.spines["top"].set_visible(False)
    ax1.spines["right"].set_visible(False)
    ax1.set_title(f"{section} params", fontsize=15)
    
    fig, ax2 = plt.subplots(figsize=figsize)
    sns.boxenplot(data=df_section, y="strategy", x="diff_release", ax=ax2)
    ax2.set_xlabel("Norm. param difference", fontsize=12)
    ax2.set_ylabel("Strategy", fontsize=12)
    ax2.spines["top"].set_visible(False)
    ax2.spines["right"].set_visible(False)
    ax2.set_title(f"{section} params - ALL", fontsize=15)

## Compare features

In [ ]:
feature_name_array = []
feature_set_array = []
feature_score_array = []
feature_type_array = []

for strategy, res in opt_results.items():
    feats = res["features"]
    for feat_name, feat_dict in feats.items():
        feature_set_array.append(strategy)
        feature_name_array.append(feat_name)
        if "MEA" not in feat_name:
            feature_type_array.append("intra")
        else:
            feature_type_array.append("extra")
        feature_score_array.append(feat_dict["score"])
        
df_feats = pd.DataFrame({"feature_set": feature_set_array, "feat_name": feature_name_array,
                         "feat_score": feature_score_array, "feature_type": feature_type_array})

In [ ]:
fig_feat_intra, ax = plt.subplots(figsize=figsize)

sns.boxplot(data=df_feats.query("feature_type == 'intra'"), y="feature_set", x="feat_score", ax=ax)
ax.set_ylabel("Feature scores (intracellular)", fontsize=12)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_title("Intracellular features", fontsize=15)

fig_feat_extra, ax = plt.subplots(figsize=figsize)

sns.boxplot(data=df_feats.query("feature_type == 'extra'"), 
              y="feature_set", x="feat_score", ax=ax)
ax.set_ylabel("Feature scores (extracellular)", fontsize=12)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_title("Extracellular features", fontsize=15)


In [ ]:
if save_fig:
    fig_feat_intra.savefig(figure_folder / "feat_intra.pdf")
    fig_feat_extra.savefig(figure_folder / "feat_extra.pdf")

## Compare EAP distance

In [ ]:
df_test

In [ ]:
fig_cos, ax = plt.subplots()
sns.barplot(data=df_test, x="strategy", y="eap_dist", ax=ax)
ax.set_ylabel("Cosine distance", fontsize=12)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_title("Extracellular difference", fontsize=15)

In [ ]:
if save_fig:
    fig_cos.savefig(figure_folder / "eap_dist.pdf")